*Это рабочий код, представленный для ознакомления.*

*Адреса файловых ресурсов скрыты,также не указаны настоящие наименование компаний-поставщиков данных.*

### Импортируем библиотеки

Увеличиваем ширину рабочей области на максимум

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

Загружаем основные библиотеки

In [2]:
#открываем необходимые библиотеки
import sys
import sqlite3 as sql3
import pandas as pd
import openpyxl
import os
import datetime
import numpy as np
from sklearn.neighbors import BallTree
#адрес БД SQLite
lct_cn = r'\\*Development\Apps\Py_projects\04_real_estate\datasets\sqlite_db\db_04_real_estate.db'
#Отключение предупреждения Pandas "SettingWithCopyWarning:A value is trying to be set on a copy of a slice from a DataFrame"
pd.options.mode.chained_assignment = None

### Проверяем справочник `geo` на наличие всех городов

In [ ]:
def unknown_cities_in_the_dictionary(geodic_q, geodic_y, sapdic_q, sapdic_y):
    #Открываем файлы
    try:
        #читаем файл словаря АД > извлекаем только 2 столбца > Удаляем дубликаты
        f1 = pd.read_csv(open(r'\\*\Apps\Py_projects\04_real_estate\datasets\dictionaries' + '\\' + str(geodic_y) + '\\04_datasets_dict_geo_0' + str(geodic_q) + '-' + str(geodic_y) + '.csv','r'), sep=';', decimal='.', usecols= ['city_sap','region_sap']).drop_duplicates().reset_index(drop=True)
        #читаем файл SAP > извлекаем только 3 столбца > делаем запрос Тип АО == "Зданию и Сооружению" и непрофиль == Нет > Выбираем 2 столбца > Удаляем дубликаты
        f2 = pd.read_excel(open(r'\\*\Real Estate\\' + str(sapdic_y) + '\SAPRE_0' + str(sapdic_q) + '-' + str(sapdic_y) + '.xlsx','rb'), sheet_name="Sheet1", usecols = ['Город','Регион','Тип АО','Непрофильный актив']).query('`Тип АО` in ("Здание") & `Непрофильный актив` == "Нет"').groupby(['Город','Регион']).agg(Count_obj=('Город','count')).reset_index()
        #Проводим сравнение
        return f1.merge(f2, left_on = ['city_sap','region_sap'], right_on = ['Город','Регион'], how = 'right').query('city_sap.isna()', engine="python")
    except OSError as e:
        print(e)

In [ ]:
unknown_cities_in_the_dictionary (geodic_q = 2, geodic_y = 2022, sapdic_q = 2, sapdic_y = 2022)

### Загрузка файла АД ПРЕДПРИЯТИЕ

In [ ]:
def load_ad_data(q,y):
    # Подключаемся к БД
    ConnectSQLite(lct_cn)
    #перед этим удаляем старые данные
    #!!! ПРОВЕРЯЙ ЧТО УДАЛЯЕШЬ, ЕСЛИ МЕНЯЕШЬ ВСЕ ДАННЫЕ, ТО НАДО УБРАТЬ УСЛОВИЕ WHERE
    ExecQuerySQLite ('DELETE FROM [ad_rent] WHERE year = ' + str(y))
    #загружаем в SQLite
    SendDataToSQLite(pd.read_csv(open(r'\\*\Apps\Py_projects\04_real_estate\datasets\main\ad' + '\\' + str(y) + '\\04_datasets_main_ad_0' + str(q) + '-' + str(y) + '.csv', 'r'), delimiter = ';'), 'ad_rent', 'append')
    # Закрытие соединения
    DisconnectSQLite()

In [ ]:
load_ad_data (2,2022)

### Расчет минимальных и максимальных сумм в разрезе городов/регионов по АД

In [ ]:
#открываем таблицу и создаем датасет ad_data_mm (min_max)
ad_data_mm = pd.read_csv(open(r'\\*\Apps\Py_projects\04_real_estate\datasets\main\ad\2022\04_datasets_main_ad_02-2022.csv', 'r'), delimiter = ';')
#создаем доп столбцы
ad_data_mm['cost_sum'] = ad_data_mm[['cost_1q','cost_2q','cost_3q','cost_4q']].sum(axis=1, min_count=1)
ad_data_mm['area_sum'] = ad_data_mm[['area_1q','area_2q','area_3q','area_4q']].sum(axis=1, min_count=1)
ad_data_mm['cost_per_1qm'] = ad_data_mm['cost_sum'] / ad_data_mm['area_sum']
#добавляем таблицу с гео-мэпингом
geo_dict = pd.read_csv(open(r'\\*\Apps\Py_projects\04_real_estate\datasets\dictionaries\2022\04_datasets_dict_geo_02-2022.csv','r'), delimiter = ';')
#добавляем признак расширенного филиала
ad_data_mm = ad_data_mm.merge(geo_dict[['city_sap','region_sap','branch_add']], left_on = ['city_sap','region_sap'], right_on = ['city_sap','region_sap'], how = 'left' )
#получаем таблицу со средними значениями до города, в т.ч. минимумы и максимумы
ad_data_mm = ad_data_mm.groupby(['year','branch_add','region_sap','city_sap','address']).agg(
    mean_cost = ('cost_per_1qm','mean')
    ).reset_index().groupby(['year','branch_add','region_sap','city_sap']).agg(
    mean_cost = ('mean_cost','mean'), min_cost = ('mean_cost','min'), max_cost = ('mean_cost','max')).reset_index()
#используем коеффициент 0.25 для уменьшения минимальной средней и максимальной средний т.е создаем коридор цен на квадратный метр
ad_data_mm[['min_cost','max_cost']]*= np.array([0.75, 1.25])

ad_data_mm.head()

### Предобработка файла НЕДВИЖИМОСТИ
На данном этапе выполняем следующее:
* фильтруем исходный файл НЕДВИЖИМОСТИ по признакам (статус, категория, тип зданя и т.д)
* заменяем районы, по Москве это Зеленоград и Троицк, а по СПБ это Сестрорецк, Петергоф и прочие
* отсеиваем данные НЕДВИЖИМОСТЬ которые не проходят по диапазонам цен за квадратный метр, а также городам присутствия

In [ ]:
def ProcessrealestateSourceFile (realestate_q, realestate_y, geodic_q, geodic_y):
    #данная функция должна использоваться с подключением к БД SQL
    #добавить район для Москва (acv_geo_dis_raion) и 
    #справочник для загрузки файла
    realestate_dict = {'realestate_id':'int64',
                     'url':'object',
                     'status_id':'int64',
                     'creationdate':'object',
                     'category_id':'int64',
                     'inputtype_id':'int64',
                     'floornumber':'int64',
                     'buildingtype_id':'int64',
                     'placementtype_id':'int64',
                     'conditiontype_id':'int64',
                     'acv_geo_region':'object',
                     'acv_geo_dis_okrug':'object',
                     'acv_geo_dis_raion':'object',
                     'acv_geo_dis_mikroraion':'object',
                     'acv_geo_gorod':'object',
                     'acv_geo_street':'object',
                     'acv_geo_house':'object',
                     'lat':'int64',
                     'lng':'int64',
                     'totalarea':'int64',
                     'bargainterms_currency':'object',
                     'bargainterms_price':'int64'}
    #справочник по геоданным
    geo_dict = pd.read_csv(open(r'\\*\Apps\Py_projects\04_real_estate\datasets\dictionaries' + '\\' + str(geodic_y) + '\\04_datasets_dict_geo_0' + str(geodic_q) + '-' + str(geodic_y) + '.csv','r'), delimiter = ';')
    #фильтры
    st_id = 1 #активные
    cat_id = 5 #аренда
    build_id = [1,2,3,4,13,17,18,19,22,28,30,31,33,34,38,42,43,44,45,46,49] #типы зданий, см.меппинг
    floor_n = 1 #этаж
    area = 40 #площадь >=
    city = list(geo_dict['city_realestate'].unique())
    region = list(geo_dict['region_realestate'].unique())
    #
    #чтение файла
    with open(r'\\*\Data\realestate\realestate_src_0' + str(realestate_q) + '-' + str(realestate_y) + '.csv','r',encoding="utf8") as t:
        t = pd.read_csv(t, usecols = list(realestate_dict.keys()))
        #изменение формата столбца для даты
        t['creationdate'] = t['creationdate'].astype('datetime64[ns]')
        #сохраняем в переменную отфильтрованные по основным фильтрам данные
        t_temp = t.query(
            '''
                ((status_id != @st_id & creationdate.dt.year==@realestate_y & creationdate.dt.quarter==@realestate_q) | (status_id == @st_id)) & \
                category_id == @cat_id & \
                ((buildingtype_id in @build_id) | (placementtype_id.notnull() & buildingtype_id.isnull())) & \
                floornumber == @floor_n & \
                totalarea >= @area & \
                acv_geo_gorod in @city & \
                bargainterms_currency == "rur"
            '''
            , engine = 'python'
            )
    #
    #делаем замену по районам, по Москве это Зеленоград и Троицк, а по СПБ это Сестрорецк, Петергоф и прочие
    #переменные по округам
    city_region = geo_dict[['city_realestate','region_realestate']].agg(''.join, axis = 1) #используется в дальнейшем как ключ
    moscow_add_okrug = ['тао (троицкий)','зелао'] #округа по полю 'acv_geo_dis_okrug', к замене
    spb_add_okrug = ['колпино','кронштадт','петергоф','пушкин','сестрорецк'] #округа по полю 'acv_geo_dis_okrug', к замене
    #переменные с условиями и результатам к условиям
    conditions = [t_temp['acv_geo_dis_okrug'].isin(moscow_add_okrug), t_temp['acv_geo_dis_mikroraion'].isin(spb_add_okrug)] #условия поиска, указываем сразу 2
    choices = [t_temp['acv_geo_dis_okrug'],t_temp['acv_geo_dis_mikroraion']] #возвращаемый результат из условий
    t_temp['acv_geo_region'] = np.select(conditions, choices, default = t_temp['acv_geo_region']) #результат с заменой
    #убираем регионы, которые не включены в справочник tGeo
    t_temp = t_temp.query('acv_geo_gorod + acv_geo_region in @city_region', engine = 'python').reset_index(drop=True)
    #добавляем города из мэпинга SAP
    t_temp[['acv_geo_gorod_map','acv_geo_region_map']] = pd.merge(
        t_temp,
        geo_dict[['city_realestate','region_realestate','city_sap','region_sap']], 
        left_on = ['acv_geo_gorod','acv_geo_region'], 
        right_on = ['city_realestate','region_realestate'],
        how = 'left'
        )[['city_sap','region_sap']]
    #отправляем в SQLite отфильтрованные данные НЕДВИЖИМОСТЬ без учета диапазона цен
    SendDataToSQLite(t_temp, 'realestate_src_0' + str(realestate_q) + '-' + str(realestate_y), 'replace')
    #отделяем диапазон между минимумом и максимумом
    t_temp = t_temp.merge( #добавляем столбцы, для определения вхождения данных НЕДВИЖИМОСТЬ в диапазон затрат на квадратный метр (на основе данных АД), стыкуемся по столбцам "_map" т.к. там учтены областные городы, типа Троицка, Зелика и прочее
        ad_data_mm[ad_data_mm['year'] == realestate_y][['city_sap','region_sap','min_cost','max_cost']], 
        left_on = ['acv_geo_gorod_map','acv_geo_region_map'], 
        right_on = ['city_sap','region_sap'], 
        how = 'inner'
    ).query( #теперь отделяем затраты и оставляем только в пределах диапазона
    'min_cost <= (bargainterms_price / totalarea) <= max_cost'
    ).drop(columns=['min_cost','max_cost','city_sap','region_sap']) #удаляем лишние столбцы
    #добавляем столбцы с годом и кварталом
    t_temp['year'] = realestate_y
    t_temp['quarter'] = realestate_q
    #отправляем данные в SQLite с учетом диапазона цен
    #..перед этим удаляем старые данные
    ExecQuerySQLite ('DELETE FROM [realestate_ads] WHERE year = ' + str(realestate_y) + ' and quarter = ' + str(realestate_q))
    #..загружаем новые
    SendDataToSQLite(t_temp, 'realestate_ads', 'append')

#### Применение функции

In [ ]:
#ограничение выгрузки
restr = [1,2022]
# Подключаемся к БД
ConnectSQLite(lct_cn)
#перебираем все года и кварталы
for y in range(2020, 2023):
    for q in range(1, 5):
        if restr[1] == y and restr[0]+1 == q: break
        print('Обработка таблицы: год - {}, квартал - {}'.format(y,q))
        ProcessrealestateSourceFile (realestate_q = q, realestate_y = y, geodic_q = 2, geodic_y = 2022)
# Закрытие соединения
DisconnectSQLite()

### Поиск альтернатив

#### Загружаем данные АД и НЕДВИЖИМОСТЬ

In [ ]:
# Подключаемся к БД
ConnectSQLite(lct_cn)
#данные НЕДВИЖИМОСТЬ
df = GetDataFromSQLite ('SELECT * FROM realestate_ads')
#данные АД ПРЕДПРИЯТИЕ
df_ad =  GetDataFromSQLite ('SELECT * FROM ad_rent')
# Закрытие соединения
DisconnectSQLite()

#### Ищем альтернативы

In [ ]:
#функция поиск альтернативных объектов в определенном радиусе
def get_alternatives_from_realestate(y, q):
    radius = 6371000 #радиус Земли
    max_distance = 1500 / radius #ограничение в поиске в метрах
    coord_1 = np.radians(df.query('year==@y & quarter == @q')[['lat','lng']].values) #коордианаты альтернативы
    coord_2 = np.radians(df_ad.query('year==@y')[['lat','lng']].values) #координаты объектов банка, к которым будем искать альтернативы
    tree = BallTree(coord_1, metric = 'haversine') #создаем дерево, по которому будем перебирать координаты
    return tree.query_radius(coord_2, r=max_distance, return_distance  = True, sort_results = True) #ищем по каждому индексу (строке) ближайшие координаты в метрах

In [ ]:
#создадим функцию, которая будет формировать таблицу из списков альтернатив
def construct_data_from_list_alternative(y,q,i):
    df_temp = pd.DataFrame(
            [
                [str(i)] * len(ind[i]),
                list(ind[i]),
                list(results[i]*radius),
                [str(list(df_ad.query('year==@y').reset_index().iloc[[i]]['id_sap'].values)[0])] * len(ind[i])
            ]
                ).T.rename(columns={0:'ad_index_alt',1:'realestate_index_alt',2:'distance_alt',3:'id_sap_alt'})
    return df.query('year==@y & quarter == @q').reset_index().iloc[ind[i]].merge(df_temp, left_index=True, right_on = 'realestate_index_alt')

In [ ]:
#ограничение выгрузки
restr = [1,2022]
#затираем итоговую таблицу перед обработкой
data_alt = pd.DataFrame()
#перебираем все года и кварталы
for y in range(2020, 2023):
    for q in range(1, 5):
        #уведомление
        print('Обработка таблицы: год - {}, квартал - {}'.format(y, q))
        #если доходим до ограничения, выходим
        if restr[1] == y and restr[0]+1 == q: break
        #индексы строк НЕДВИЖИМОСТЬ и результат в расстоянии
        ind, results = get_alternatives_from_realestate (y, q)
        #перебираем все строки таблицы АД и ищем им альтернативы
        for i in range(len(df_ad.query('year == @y'))):
            if list(ind[i]):
                #формируем временную таблицу с данными НЕДВИЖИМОСТЬ соседями
                df_temp = construct_data_from_list_alternative (y, q, i)
                #оставляем только те объявления, при которых координаты и площадь одинаковые, но создан последним (неочевидные дубликаты)
                df_temp = df_temp[df_temp['creationdate'].groupby(df_temp[['lat','lng','totalarea']].astype(str).apply('_'.join, 1)).apply(lambda x: x == x.max())]
                #оставляем только те объявления, при которых координаты и площадь одинаковые, при этом только наиболее дешевые будут
                df_temp = df_temp[df_temp['bargainterms_price'].groupby(df_temp[['lat','lng','totalarea']].astype(str).apply('_'.join, 1)).apply(lambda x: x == x.min())]
                #удаляем очевидные дубликаты, если имеются одинаковые цены
                df_temp = df_temp.drop_duplicates(subset=['lat','lng','totalarea','bargainterms_price'])
                #отсеиваем площади, смотрим какая площадь на объекте в АД и подбираем аналоги в пределах +/- 25%
                ad_obj_area = list(df_ad.query('year==@y').reset_index().iloc[[i]]['area_' + str(q) + 'q'].values)[0]
                df_temp = df_temp.query('(@ad_obj_area*0.75) <= totalarea <= (@ad_obj_area*1.25)')
                #заполняем сводный файл
                if data_alt.empty:
                    data_alt = df_temp
                else:
                    data_alt = pd.concat([data_alt, df_temp])
                #очищаем
                df_temp = None
#удалеяем столбцы с индексами, которые уже нам не нужны
data_alt = data_alt.drop(columns=['ad_index_alt','realestate_index_alt','index']) #p.s столбец index необходимо удалить в самом начале методом drop = True

In [ ]:
#ОТПРАВКА В SQLITE
# Подключаемся к БД
ConnectSQLite(lct_cn)
#перед этим удаляем старые данные
#ExecQuerySQLite ('DELETE FROM [realestate_alt]')
#загружаем в SQLite
SendDataToSQLite(data_alt, 'realestate_alt', 'append')
# Закрытие соединения
DisconnectSQLite()

### Формирование итоговой таблицы
Таблица realestate_total будет использоваться во всех расчетах, кроме поиска альтернатив

In [ ]:
#СОЗДАЕМ ИТОГОВУЮ ТАБЛИЦУ ПО АД ПРЕДПРИЯТИЕ
# Подключаемся к БД
ConnectSQLite(lct_cn)
# получаем данные
df = GetDataFromSQLite('SELECT * FROM ad_rent')
#отключаемся от базы
DisconnectSQLite()
#ПЕРЕВОРОТ СТОЛБЦОВ В СТРОКИ
#--временная таблица
df_temp = df[['year','id_sap','address','district','city_sap','region_sap','cost_1q', 'cost_2q','cost_3q','cost_4q','area_1q', 'area_2q','area_3q','area_4q']]
#--переворот столбцов в строки по суммам
df_temp_1 = df_temp.melt(id_vars=['year','id_sap','address','district','city_sap','region_sap'], value_vars=['cost_1q', 'cost_2q','cost_3q','cost_4q'], var_name = 'quarter', value_name='price')
df_temp_1['quarter']=df_temp_1['quarter'].str.extract('(\d+)')
#--переворот столбцов в строки по площадям
df_temp_2 = df_temp.melt(id_vars=['year','id_sap','address','district','city_sap','region_sap'], value_vars=['area_1q', 'area_2q','area_3q','area_4q'], var_name = 'quarter', value_name='area')
df_temp_2['quarter']=df_temp_2['quarter'].str.extract('(\d+)')
#--соединение временных таблиц в одну
df_temp_1 = df_temp_1.merge(df_temp_2[['year','id_sap','quarter','area']], how='outer',on=['year','id_sap','quarter'])
#--удаляем строки, если `price` нулевой, это касается ПРЕДПРИЯТИЕ т.к. делаем разворот из столбцов в строки
df_temp_1 = df_temp_1[~df_temp_1['price'].isnull()].reset_index(drop=True)
#цепляем справочник tGeo
df_temp_1 = df_temp_1.merge(geo_dict[['city_sap','region_sap','branch_add','oap']], left_on = ['city_sap','region_sap'], right_on = ['city_sap','region_sap'], how = 'left' )
#меняем названия столбцов
df_temp_1.rename(columns={'city_sap':'city','region_sap':'region','id_sap_alt':'id_sap','oap':'pap','branch_add':'branch'}, inplace = True)
#добавляем дополнительный столбец с признаком "ГО/Регион"
df_temp_1['branch_group'] = ''
df_temp_1.loc[df_temp_1['branch'].str.contains('ГО'),'branch_group'] = 'Москва и МО'
df_temp_1.loc[~df_temp_1['branch'].str.contains('ГО'),'branch_group'] = 'Регионы'
#добавляем расчетный столбец med_sqm
df_temp_1['med_sqm'] = df_temp_1['price'] / df_temp_1['area']
#!!! ТУТ МОЖНО ПЕРЕДАТЬ КУСОК В ФУНКЦИЮ, НЕ СДЕЛАЛ Т.К. ПО ВРЕМЕНИ НЕ УСПЕВАЛ
#расчетные столбцы
cols = ['district','city','pap','branch','branch_group','total'] #колонки, которые будем добавлять
for c in cols:
    #определяем столбцы для временной таблицы 1 и объединения 2
    df_gr_cols_1 = ['year','quarter'] if c=='total' else ['year','quarter',c]
    df_gr_cols_2 = ['year','quarter','med_'+c] if c=='total' else ['year','quarter',c,'med_'+c]
    #формируем временную таблицу
    df_temp = df_temp_1.groupby(df_gr_cols_1)['med_sqm'].median().reset_index()
    df_temp.rename(columns={'med_sqm':'med_'+c}, inplace=True)
    #добавляем в итоговую таблицу
    df_temp_1 = df_temp_1.merge(df_temp[df_gr_cols_2], how='outer',on=df_gr_cols_1)
    df_temp = None
#--признак среза
df_temp_1['source'] = 'company'
#--пересортировка
df_temp_1 = df_temp_1 [['source','year','quarter', 'id_sap', 'address', 'district', 'city', 'region','branch', 'pap', 'branch_group','price', 'area','med_sqm', 'med_district', 'med_city', 'med_pap', 'med_branch','med_branch_group', 'med_total']]
#очищаем временную и начальную таблицу
df_temp = None
df_temp_2 = None
df = None

In [ ]:
#СОЗДАЕМ ИТОГОВУЮ ТАБЛИЦУ ПО НЕДВИЖИМОСТЬ
# Подключаемся к БД
ConnectSQLite(lct_cn)
# получаем данные
df = GetDataFromSQLite('SELECT * FROM realestate_alt')
df_ad = GetDataFromSQLite('SELECT year, id_sap, address, district FROM ad_rent')
#отключаемся от базы
DisconnectSQLite()
#создаем столбец с расчетом затрат на квадратный метр
df['sqm'] = df['bargainterms_price'] / df['totalarea']
#формируем таблицу с подготовленными медианными значениями
df_alt = df.groupby(['year','quarter','id_sap_alt','acv_geo_gorod_map','acv_geo_region_map']).agg(med_sqm=('sqm','median')).reset_index()
#цепляем справочник tGeo
df_alt = df_alt.merge(geo_dict[['city_sap','region_sap','branch_add','oap']], left_on = ['acv_geo_gorod_map','acv_geo_region_map'], right_on = ['city_sap','region_sap'], how = 'left' ).drop(columns=['city_sap','region_sap'])
#добавим район из таблицы ad_rent
df_alt = df_alt.merge(df_ad, left_on = ['year','id_sap_alt'], right_on = ['year','id_sap'], how = 'left' ).drop(columns=['id_sap'])
#корректируем район, если это Москва или Питер, то оставляем, если нет, то пишем вместо пустоты или района Город !!! убрал т.к. формируется на этапе Excel в файле
#df_alt.loc[~df_alt['acv_geo_gorod_map'].isin(['Москва г','Санкт-Петербург г']),'district'] = df_alt.loc[~df_alt['acv_geo_gorod_map'].isin(['Москва г','Санкт-Петербург г']),'acv_geo_gorod_map']

#меняем названия столбцов
df_alt.rename(columns={'acv_geo_gorod_map':'city','acv_geo_region_map':'region','id_sap_alt':'id_sap','oap':'pap','branch_add':'branch'}, inplace = True)

#добавляем дополнительный столбец с признаком "ГО/Регион"
df_alt['branch_group'] = ''
df_alt.loc[df_alt['branch'].str.contains('ГО'),'branch_group'] = 'Москва и МО'
df_alt.loc[~df_alt['branch'].str.contains('ГО'),'branch_group'] = 'Регионы'
#расчетные столбцы
cols = ['district','city','pap','branch','branch_group','total'] #колонки, которые будем добавлять
for c in cols:
    #определяем столбцы для временной таблицы 1 и объединения 2
    df_gr_cols_1 = ['year','quarter'] if c=='total' else ['year','quarter',c]
    df_gr_cols_2 = ['year','quarter','med_'+c] if c=='total' else ['year','quarter',c,'med_'+c]
    #формируем временную таблицу
    df_temp = df_alt.groupby(df_gr_cols_1)['med_sqm'].median().reset_index()
    df_temp.rename(columns={'med_sqm':'med_'+c}, inplace=True)
    #добавляем в итоговую таблицу
    df_alt = df_alt.merge(df_temp[df_gr_cols_2], how='outer',on=df_gr_cols_1)
    df_temp = None
#--признак среза
df_alt['source'] = 'realestate'
#--пересортировка
df_alt = df_alt[['source','year', 'quarter', 'id_sap', 'address','district', 'city', 'region', 'branch',  'pap',   'branch_group','med_sqm', 'med_district','med_city', 'med_pap', 'med_branch', 'med_branch_group','med_total']]

In [ ]:
#подключение к базе
ConnectSQLite(lct_cn)
#загрузка данных Предприятия
ExecQuerySQLite('DELETE FROM realestate_total WHERE source="company"')
SendDataToSQLite(df_temp_1, 'realestate_total', 'append')
#загрузка данных НЕДВИЖИМОСТЬ
ExecQuerySQLite('DELETE FROM realestate_total WHERE source="realestate"')
SendDataToSQLite(df_alt, 'realestate_total', 'append')
#создадим файл Excel для Access
df_ttl = GetDataFromSQLite('SELECT * FROM realestate_total')
df_ttl.to_excel(r'\\*\Apps\Py_projects\04_real_estate\datasets\main\ad\2022_Q2_ttl.xlsx', sheet_name = f'realestate_total', encoding = 'windows-1251', index = False)                
#отключаемся от БД
DisconnectSQLite()

### Дополнительные функции

Функция для расчета ближайший точек

In [ ]:
def NearestPoints(q,y):
    # Файлы загрузки и сохранения
    # - DH_Rent_ForSQLite.csv (данные с адресами АД аренды + искуственно наложенный мэппинг по городам и координатам в аналогичном Excel файле)
    # - realestate_Source (данные из SQLite, которые были обработаны и отфильтрованы согласно требованиям)
    # - realestate_Uniq (сохраняемый файл, где отображены уникальные строки по ключевым столбцам 'lat'+'lng'+'totalarea'+'Converted_Price'
    # Переменные
    # - "y" - Фильтр дла данных DH_Rent_ForSQLite, файл ad_realestate_csv будет браться с уже определенным кварталом; также для указания года использованных данных (DH_Rent_ForSQLite и ad_realestate_csv) в конечном файле
    # - "q" - для указания в конечном файле какой квартал использовался ad_realestate_csv
    # Загружаем CSV
    ad_csv = None
    realestate_csv = None
    ad_csv = pd.read_csv(open(r'\\*\Apps\Market Rental Rate Report\csv\DH_Rent_ForSQLite.csv','r'), sep = ';', decimal = '.') #!!! проверяй в координатах, чтобы не было точек т.к. будет восприниматься как текст
    ad_csv = ad_csv.loc[ad_csv['Тип ОН']=='Точка продаж'].reset_index(drop=True)
    ad_csv = ad_csv.loc[ad_csv['Год']==y].drop_duplicates(subset=['Адрес']).reset_index(drop=True)
    # открываем realestate
    realestate_csv = pd.read_csv (open(r'\\*\Apps\Market Rental Rate Report\csv\realestate_filtered\realestate_fltrd_0' + str(q) + '-' + str(y) +'.csv','r'), sep=';', decimal = '.'
        ,usecols = ['realestate_id','url','buildingtype_id','conditiontype_id','Филиал','ФилиалДоп','ПАП','acv_geo_region','acv_geo_gorod','street','lat','lng','totalarea','Converted_Price'])
    #Оставляем в realestate_CSV только города, которые есть в ad_csv т.е. только в городах присутствия
    arr_cities = ad_csv.drop_duplicates(subset=['Город'])['Город'].values
    realestate_csv = realestate_csv.query('acv_geo_gorod in @arr_cities').reset_index(drop=True)
    arr_cities = None
    # Добавляем пользовательские столбцы
    realestate_csv = realestate_csv.assign(**{'distance': 0, 'c_lat': 0, 'c_lng': 0, 'adr': 0, 'sort_id': 0, 'quarter': q, 'year': y})
    #форматы столбцов
    df_realestate_types = {'year':'int64', 'quarter':'int64','adr':'object', 'sort_id':'int64', 'c_lng':'float', 'c_lat':'float', 'distance':'float', 'realestate_id':'int64', 'url':'object','buildingtype_id':'object','conditiontype_id':'object', 'Филиал':'object', 'ФилиалДоп':'object', 'ПАП':'object', 'acv_geo_region':'object'
        ,'acv_geo_gorod':'object','street':'object', 'lat':'float64', 'lng':'float64', 'totalarea':'float64', 'Converted_Price':'float64'}
    # Меняем столбцы на числовой формат
    realestate_csv.astype(df_realestate_types).dtypes
    # Удаляем из таблицы строки, по которым Ключ: Широта+Долгота+Площадь больше минимальной строки с таким же ключом т.к. из дубликатов оставляем минимальный по сумме Converted_Price
    realestate_csv = realestate_csv[realestate_csv['Converted_Price'].groupby(realestate_csv[['lat','lng','totalarea']].astype(str).apply('_'.join, 1)).apply(lambda x: x == x.min())]
    # Удаляем дубликаты по тому же ключу т.к. если минимальные оставались одинаковые, то строки оставались без удаления
    realestate_csv = realestate_csv.drop_duplicates(subset=['lat','lng','totalarea','Converted_Price'])
    # - сохраняем в CSV и отправляем в SQLute
    realestate_csv.to_csv(r'\\*\Apps\Market Rental Rate Report\csv\realestate_Uniq_0' + str(q) + '-' + str(y) +'.csv', sep=';', encoding='windows-1251', index = False)
    UploadCSV2SQLite ('\db_MRRP.db', 'DELETE FROM [realestate_ads] WHERE year = ' + str(y) + ' and quarter = ' + str(q),'\realestate_Uniq_0' + str(q) + '-' + str(y) +'.csv','realestate_ads')
    # Создаем подготовленный Dataframe для вывода результата
    df_np = None
    df_np = pd.DataFrame(columns=list(df_realestate_types.keys()))
    # устанавливаем форматы столбцов
    df_np.astype(df_realestate_types).dtypes
    # Перебираем построчно данные АД и добавляем в таблицу итоговые данные
    realestate_t = None
    for r in ad_csv.itertuples():
        # временная таблица
        avg_sqr = ad_csv.iloc[r.Index, 9:13].mean()
        realestate_t = realestate_csv.loc[(realestate_csv['acv_geo_gorod'] == ad_csv.iloc[r.Index,18]) & (realestate_csv['totalarea'] >= avg_sqr * 0.70) & (realestate_csv['totalarea'] <= avg_sqr * 1.3)]
        # проверяем есть ли данные с таким городом в realestate_t, если нет, то идем дальше
        if not realestate_t.empty:
            # меняем в таблице realestate_CSV данные в столбцах сравниваемых координат, указываем координату от которой будем считать расстояние
            realestate_t['year'] = int(y)
            realestate_t['adr'] = ad_csv.iloc[r.Index,3]
            realestate_t['c_lat'] = ad_csv.iloc[r.Index,21]
            realestate_t['c_lng'] = ad_csv.iloc[r.Index,22]
            # рассчитываем расстояние
            realestate_t['distance'] = haversine(realestate_t.loc[0:,'c_lat'],realestate_t.loc[0:,'c_lng'],realestate_t.loc[0:,'lat'],realestate_t.loc[0:,'lng'])
            # делаем сортировку
            realestate_t = realestate_t.sort_values(by = ['distance'], ignore_index = True)
            # указываем индекс
            realestate_t['sort_id'] = realestate_t.index
            # вставляем полученние данные в новую таблицу
            df_np = df_np.append(pd.DataFrame(realestate_t.iloc[:5,:], columns = realestate_t.columns))
            #df_np = df_np.append(pd.DataFrame(realestate_t.iloc[:,:], columns = realestate_t.columns))
            # Очищаем временную
            realestate_t = None
            avg_sqr = None
    # сохраняем в CSV и SQLite
    df_np.to_csv(r'\\*\Apps\Market Rental Rate Report\csv\realestate_TOP5_0' + str(q) + '-' + str(y) + '.csv', sep = ';', encoding = 'windows-1251', index = False)
    UploadCSV2SQLite ('\db_MRRP.db', 'DELETE FROM [realestate_top5] WHERE year = ' + str(y) + ' and quarter = ' + str(q),'\realestate_TOP5_0' + str(q) + '-' + str(y) + '.csv','realestate_top5') #top-5
    # очищаем переменные
    del ad_csv
    del realestate_csv
    del df_np
    del realestate_t

Функция для загрузки файлов CSV в SQLite

In [ ]:
def UploadCSV2SQLite(lct, del_qr, file, table_sql):
    # Подключаемся к БД
    lct_cn = r'\\*\Apps\Market Rental Rate Report\db' +  lct
    ConnectSQLite(lct_cn)
    # Очищаем данные перед загрузкой
    ExecQuerySQLite(del_qr)
    # Отправка файлов
    # отправляемый файл
    file_v = pd.read_csv(open(r'\\*\Apps\Market Rental Rate Report\csv' + file,'r'), sep = ';', decimal = ',')
    # отправка
    SendDataToSQLite(file_v, table_sql, 'append')
    # очищаем переменные
    file_v = None
    table_v  = None
    # Закрытие соединения
    DisconnectSQLite()
    lct_cn = None

Функция для сохранения файлов CSV в Excel для загрузки в Access и её выполнение

In [ ]:
#создание функции
def SaveForAccess(name_v):
    #!Автоматизиовать процесс!
    # Сохраняет обработаный файл MRRR_Main из SQLite для загрузки в Access т.к. разнятся кодировки и разделительный символ
    ad_csv = pd.read_csv (open(r'\\*\Apps\Market Rental Rate Report\csv\realestate_filtered\\' + name_v + '.csv','r'), sep = ';', decimal='.')
    writer = pd.ExcelWriter(r'\\*\Apps\Market Rental Rate Report\csv\2access\\' + name_v + '.xlsx',engine='xlsxwriter',options={'strings_to_urls': False})
    ad_csv.to_excel(writer, sheet_name = name_v, encoding = 'windows-1251', index = False)
    writer.save()
    ad_csv = None
#выполнение
files_arr = ['realestate_main','realestate_total','realestate_top5','realestate_ads','realestate_ad','realestate_description']
for v in files_arr:
    SaveForAccess (v)

### Запросы SQL для обработки

#### Функции для SQL

In [ ]:
cn = None
def ConnectSQLite(dbLoc):
#Соединение к БД SQLite
    try:
        global cn
        cn = None
        cn = sql3.connect(str(dbLoc))
    except sql3.Error as error:
        print("SQLite Error: ", error)
    finally:
        if (cn):
            print("Successfully connected!")

def DisconnectSQLite():
#Соединение к БД SQLite
    try:
        global cn
        cn.close()
        cn = None
    except sql3.Error as error:
        print("SQLite Error: ", error)
    finally:
        if not (cn):
            print("Successfull disconnected!")
        
def GetDataFromSQLite (qr):
#Получение данных из запроса к SQLite
    try:
        vData = pd.io.sql.read_sql(qr, cn)
        return vData
    except sql3.Error as error:
        print("SQLite Error: ", error)
    finally:
        if (cn):
            print("Query completed successfully!")
        
def ExecQuerySQLite (qr):
#Выполнить запрос SQL
    try:
        cur = cn.cursor()
        cur.execute(qr)
        cn.commit()
    except sql3.Error as error:
        print("SQLite Error: ", error)
    finally:
        if (cn):
            print("Execution query completed successfully!")
            
def SendDataToSQLite (dfIn,tabInOut,ifex):
#Вставить данные в SQLite из листа Excel с помощью Pandas
    try:
        err_msg = None
        #Вынести отдельно tData = pd.read_excel(pathFile,sheet_name=tabInOut)
        dfIn.to_sql(tabInOut, cn, if_exists = ifex, index = False)
    except sql3.Error as error:
        err_msg = error
        print("Error: ", err_msg)
    finally:
        if err_msg == None and (cn):
            print("Append completed successfully!")
        else:
            print("Append failed!")